In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
from collections import Counter, defaultdict
from PIL import Image

In [3]:
# TODO: PATH 설정

# main project path
PROJECT_ROOT = Path('/content/drive/Othercomputers/내 노트북/lion_final_pro_multimodal-anomaly-report-generation')
os.chdir(PROJECT_ROOT) # 현재 경로 수정
# print(f"현재 작업 경로: {os.getcwd()}")

# dataset path
DATA_ROOT = PROJECT_ROOT / "dataset" / "MMAD"
DOMAIN_JSON = DATA_ROOT / "domain_knowledge.json"
MMAD_JSON = DATA_ROOT / "mmad.json"
META_CSV = DATA_ROOT / "metadata.csv"

# config.yaml path
CONFIG_ROOT = PROJECT_ROOT / "configs"
RUNTIME_CONFIG_ROOT = CONFIG_ROOT / "runtime.yaml"
EVAL_CONFIG_ROOT = CONFIG_ROOT / "eval.yaml"

# check (선택사항)
print(f"Project Root: {PROJECT_ROOT}")
print(f"Data Root: {DATA_ROOT}")
print(f"Config Root: {CONFIG_ROOT}")
print(f"Current working directory: {os.getcwd()}")

Project Root: /content/drive/Othercomputers/내 노트북/lion_final_pro_multimodal-anomaly-report-generation
Data Root: /content/drive/Othercomputers/내 노트북/lion_final_pro_multimodal-anomaly-report-generation/dataset/MMAD
Config Root: /content/drive/Othercomputers/내 노트북/lion_final_pro_multimodal-anomaly-report-generation/configs
Current working directory: /content/drive/Othercomputers/내 노트북/lion_final_pro_multimodal-anomaly-report-generation


In [4]:
# GPU/시스템 정보 확인
%%bash
echo "========================================================================"
echo "🖥️ 시스템 정보"
echo "========================================================================"
echo ""
echo "📅 현재 시간: $(date '+%Y-%m-%d %H:%M:%S')"
echo ""
echo "🔧 GPU 정보:"
nvidia-smi --query-gpu=name,memory.total,driver_version --format=csv,noheader
echo ""
echo "🔧 CUDA 버전:"
nvcc --version 2>/dev/null | grep "release" || echo "nvcc not found"
echo ""
echo "🔧 Python 버전:"
python --version
echo ""
echo "🔧 디스크 공간:"
df -h /content | tail -1 | awk '{print "   사용: " $3 " / " $2 " (" $5 " 사용중)"}'
echo ""
echo "========================================================================"

🖥️ 시스템 정보

📅 현재 시간: 2026-01-31 13:39:02

🔧 GPU 정보:
Tesla T4, 15360 MiB, 550.54.15

🔧 CUDA 버전:
Cuda compilation tools, release 12.5, V12.5.82

🔧 Python 버전:
Python 3.12.12

🔧 디스크 공간:
   사용: 39G / 236G (17% 사용중)



In [5]:
# Miniconda + UV 설치
%%bash
set -e

echo "========================================================================"
echo "📦 Miniconda + UV 설치"
echo "========================================================================"
echo "📅 시작: $(date '+%H:%M:%S')"
echo ""

# Miniconda 설치
if [ ! -d "/content/conda" ]; then
    echo "1️⃣ Miniconda 다운로드 및 설치..."
    wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O /tmp/miniconda.sh
    bash /tmp/miniconda.sh -b -p /content/conda > /dev/null 2>&1
    rm /tmp/miniconda.sh
    echo "   ✅ Miniconda 설치 완료"
else
    echo "1️⃣ Miniconda 이미 설치됨"
fi

# UV 설치 (base 환경에)
if [ ! -f "/content/conda/bin/uv" ]; then
    echo "2️⃣ UV 설치..."
    /content/conda/bin/pip install uv -q
    echo "   ✅ UV 설치 완료"
else
    echo "2️⃣ UV 이미 설치됨"
fi

echo ""
echo "📋 UV 버전:"
/content/conda/bin/uv --version

echo ""
echo "📅 완료: $(date '+%H:%M:%S')"
echo "========================================================================"

📦 Miniconda + UV 설치
📅 시작: 13:39:02

1️⃣ Miniconda 다운로드 및 설치...
   ✅ Miniconda 설치 완료
2️⃣ UV 설치...
   ✅ UV 설치 완료

📋 UV 버전:
uv 0.9.28

📅 완료: 13:39:27


In [6]:
# Anaconda ToS 동의 + Python 3.10 conda 환경 생성
%%bash
set -e

echo "========================================================================"
echo "🐍 Python 3.10 conda 환경 생성"
echo "========================================================================"
echo "📅 시작: $(date '+%H:%M:%S')"
echo ""

source /content/conda/bin/activate

# Anaconda Terms of Service 동의
echo "1️⃣ Anaconda Terms of Service 동의..."
conda config --set solver classic 2>/dev/null || true
echo "y" | conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main 2>/dev/null || true
echo "y" | conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r 2>/dev/null || true
conda config --add channels conda-forge 2>/dev/null || true
conda config --set channel_priority flexible 2>/dev/null || true
echo "   ✅ 설정 완료"
echo ""

if [ ! -d "/content/conda/envs/anomaly_report" ]; then
    echo "2️⃣ anomaly_report 환경 생성 중..."
    conda create -n anomaly_report python=3.10 -c conda-forge -y -q 2>&1 | tail -5
    echo "   ✅ 환경 생성 완료"
else
    echo "2️⃣ anomaly_report 환경 이미 존재"
fi

echo ""
echo "📋 Python 버전 확인:"
/content/conda/envs/anomaly_report/bin/python --version
echo ""
echo "📅 완료: $(date '+%H:%M:%S')"
echo "========================================================================"

🐍 Python 3.10 conda 환경 생성
📅 시작: 13:39:27

1️⃣ Anaconda Terms of Service 동의...
accepted Terms of Service for https://repo.anaconda.com/pkgs/main
accepted Terms of Service for https://repo.anaconda.com/pkgs/r
   ✅ 설정 완료

2️⃣ anomaly_report 환경 생성 중...


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
   ✅ 환경 생성 완료

📋 Python 버전 확인:
Python 3.10.19

📅 완료: 13:40:10


In [7]:
# NumPy 1.26.4 설치
%%bash
set -e

echo "========================================================================"
echo "📦 NumPy 1.26.4 설치"
echo "========================================================================"
echo "📅 시작: $(date '+%H:%M:%S')"
echo ""

PY=/content/conda/envs/anomaly_report/bin/python
UV=/content/conda/bin/uv

echo "1️⃣ NumPy 설치 중..."
$UV pip install numpy==1.26.4 --python $PY -q

echo ""
echo "📋 NumPy 버전 확인:"
$PY -c "import numpy; print(f'   NumPy: {numpy.__version__}')"
echo ""
echo "📅 완료: $(date '+%H:%M:%S')"
echo "========================================================================"

📦 NumPy 1.26.4 설치
📅 시작: 13:40:10

1️⃣ NumPy 설치 중...

📋 NumPy 버전 확인:
   NumPy: 1.26.4

📅 완료: 13:40:12


In [8]:
# PyTorch 2.1.2 + CUDA 11.8 설치
%%bash
set -e

echo "========================================================================"
echo "🔥 PyTorch 2.1.2 + CUDA 11.8 설치"
echo "========================================================================"
echo "📅 시작: $(date '+%H:%M:%S')"
echo ""

PY=/content/conda/envs/anomaly_report/bin/python
UV=/content/conda/bin/uv

echo "1️⃣ PyTorch 설치 중 (약 1분 소요)..."
$UV pip install torch==2.1.2 torchvision==0.16.2 \
    --index-url https://download.pytorch.org/whl/cu118 \
    --python $PY -q

echo ""
echo "📋 설치 확인:"
$PY -c "
import torch
print(f'   PyTorch: {torch.__version__}')
print(f'   CUDA 사용 가능: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'   GPU: {torch.cuda.get_device_name(0)}')
    print(f'   CUDA 버전: {torch.version.cuda}')
"

echo ""
echo "📅 완료: $(date '+%H:%M:%S')"
echo "========================================================================"

🔥 PyTorch 2.1.2 + CUDA 11.8 설치
📅 시작: 13:40:12

1️⃣ PyTorch 설치 중 (약 1분 소요)...

📋 설치 확인:
   PyTorch: 2.1.2+cu118
   CUDA 사용 가능: True
   GPU: Tesla T4
   CUDA 버전: 11.8

📅 완료: 13:41:03


In [11]:
# Anomalib 1.1.0 설치 + 필수 라이브러리 설치
%%bash
set -e

echo "========================================================================"
echo "🚀 Anomalib 1.1.0 설치 최종 확인"
echo "========================================================================"

PY=/content/conda/envs/anomaly_report/bin/python
UV=/content/conda/bin/uv

# 1. 설치는 이미 되었으므로 확인 과정만 안전하게 수행
echo "📋 패키지 버전 최종 점검 (Backend 무시 설정):"
echo "------------------------------------------------------------------------"

# MPLBACKEND를 Agg(비화면 출력용 기본 백엔드)로 설정하여 에러 방지
export MPLBACKEND=Agg

$PY -c "
import pandas
import tqdm
import anomalib
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

print(f'✅ Pandas:     {pandas.__version__}')
print(f'✅ Matplotlib: {matplotlib.__version__}')
print(f'✅ TQDM:       {tqdm.__version__}')
print(f'✅ Anomalib:   {anomalib.__version__}')
"

echo ""
echo "📅 완료: $(date '+%H:%M:%S')"
echo "========================================================================"

🚀 Anomalib 1.1.0 설치 최종 확인
📋 패키지 버전 최종 점검 (Backend 무시 설정):
------------------------------------------------------------------------
✅ Pandas:     2.2.2
✅ Matplotlib: 3.10.8
✅ TQDM:       4.67.2
✅ Anomalib:   1.1.0

📅 완료: 13:48:22


In [14]:
%%bash
set -e

PY=/content/conda/envs/anomaly_report/bin/python
UV=/content/conda/bin/uv

echo "========================================================================"
echo "🔧 NumPy 버전 복구 및 최종 라이브러리 점검"
echo "========================================================================"

# 1. NumPy 1.26.4로 강제 강등 (충돌 해결의 핵심)
echo "1️⃣ NumPy 1.26.4 재설정 중..."
$UV pip install "numpy==1.26.4" --python $PY -q --force-reinstall

# 2. 나머지 라이브러리 재확인 설치
echo "2️⃣ 필수 라이브러리 최종 동기화..."
$UV pip install \
    "lightning>=2.1.0,<2.2.0" \
    "torchmetrics>=1.2.0,<1.3.0" \
    "opencv-python-headless" \
    "scikit-learn" \
    "scikit-image" \
    "seaborn" \
    --python $PY -q

echo ""
echo "📋 최종 설치 결과 확인:"
echo "------------------------------------------------------------------------"

# 백엔드 에러 방지를 위해 Agg 설정 주입
export MPLBACKEND=Agg

$PY -c "
import os
import numpy
import torch
import lightning
import torchmetrics
import cv2
import sklearn
import seaborn
import matplotlib
matplotlib.use('Agg')

print(f'✅ NumPy:         {numpy.__version__}')
print(f'✅ PyTorch:       {torch.__version__}')
print(f'✅ Lightning:     {lightning.__version__}')
print(f'✅ Torchmetrics:  {torchmetrics.__version__}')
print(f'✅ OpenCV:        {cv2.__version__}')
print(f'✅ Scikit-Learn:  {sklearn.__version__}')
print(f'✅ Seaborn:       {seaborn.__version__}')
"
echo "------------------------------------------------------------------------"
echo "✨ 모든 충돌이 해결되었습니다! 이제 학습 준비 완료입니다."
echo "========================================================================"

🔧 NumPy 버전 복구 및 최종 라이브러리 점검
1️⃣ NumPy 1.26.4 재설정 중...
2️⃣ 필수 라이브러리 최종 동기화...

📋 최종 설치 결과 확인:
------------------------------------------------------------------------
✅ NumPy:         1.26.4
✅ PyTorch:       2.1.2+cu118
✅ Lightning:     2.1.4
✅ Torchmetrics:  1.2.1
✅ OpenCV:        4.11.0
✅ Scikit-Learn:  1.7.2
✅ Seaborn:       0.13.2
------------------------------------------------------------------------
✨ 모든 충돌이 해결되었습니다! 이제 학습 준비 완료입니다.


/content/conda/envs/anomaly_report/lib/python3.10/site-packages/lightning/fabric/__init__.py:40: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.


In [15]:
import subprocess

# 가상환경 내 파이썬 경로
PY = "/content/conda/envs/anomaly_report/bin/python"

# 확인할 패키지 목록
target_packages = [
    "anomalib", "lightning", "torchmetrics",
    "numpy", "torch", "torchvision", "opencv-python", "pillow", "scikit-learn", "scikit-image",
    "pyyaml", "tqdm", "matplotlib", "seaborn", "pandas"
]

print("========================================================================")
print("🧐 anomaly_report 환경 내부 패키지 점검 결과")
print("========================================================================")

# pip list를 통해 설치된 패키지 확인
try:
    result = subprocess.check_output([PY, "-m", "pip", "list"], text=True)
    installed_list = result.lower()

    for pkg in target_packages:
        # opencv는 패키지명이 opencv-python이므로 체크 방식 조정
        check_name = "opencv-python" if pkg == "opencv-python" else pkg.lower()

        if check_name in installed_list:
            # 실제 설치된 버전 행 추출
            version_line = [line for line in result.split('\n') if line.lower().startswith(check_name)][0]
            print(f"✅ {version_line}")
        else:
            print(f"❌ {pkg} : 설치되지 않음")

except Exception as e:
    print(f"에러 발생: {e}")

print("========================================================================")

🧐 anomaly_report 환경 내부 패키지 점검 결과
✅ anomalib               1.1.0
✅ lightning              2.1.4
✅ torchmetrics           1.2.1
✅ numpy                  1.26.4
✅ torch                  2.1.2+cu118
✅ torchvision            0.16.2+cu118
✅ opencv-python-headless 4.11.0.86
✅ pillow                 12.0.0
✅ scikit-learn           1.7.2
✅ scikit-image           0.25.2
✅ PyYAML                 6.0.3
✅ tqdm                   4.67.2
✅ matplotlib             3.10.8
✅ seaborn                0.13.2
✅ pandas                 2.2.2
